In [11]:
import csv
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from datetime import datetime

In [2]:
pd.options.display.max_columns = None #Displays all columns

In [3]:
def nucleated(x):
    nuc_high_seal = x[(x['Post patch?'] == 'Nucleated') | 
                      (((x['Post patch?'] == 'nucleus_visible') | 
                      (x['Post patch?'] == 'nucleus_present')) & 
                      (x['Post patch pipette R'] >= 500))]
    return nuc_high_seal

def partial_nucleated(y):
    nuc_low_seal = y[(y['Post patch?'] == 'Partial-Nucleus') | 
                     (((y['Post patch?'] == 'nucleus_present') | 
                     (y['Post patch?'] == 'nucleus_visible')) & 
                     (y['Post patch pipette R'] <= 499))]
    return nuc_low_seal

def outside_out(z):
    no_high_seal = z[(z['Post patch?'] == 'Outside-Out') | 
                     (((z['Post patch?'] == 'nucleus_absent') | 
                     (z['Post patch?'] == 'no_nucleus_visible')) & 
                     (z['Post patch pipette R'] >= 500))]
    return no_high_seal

def no_seal(w): 
    no_low_seal = w[(w['Post patch?'] == 'No-Seal') | 
                    (((w['Post patch?'] == 'nucleus_absent') | 
                    (w['Post patch?'] == 'no_nucleus_visible')) & 
                    (w['Post patch pipette R'] <= 499))]
    return no_low_seal
    
def entire_cell(v):
    entire = v[(v['Post patch?'] == 'Entire-Cell') | 
               (v['Post patch?'] == 'entire_cell')]
    return entire

#variable['post_patch'] = 'Term'
#Term is an output displayed in the Post_Patch column

def reclassify(df):
    nu = nucleated(df)
    nu['post_patch'] = 'Nuc-high seal' 
    oo = outside_out(df)
    oo['post_patch'] = 'No-high seal'
    pn = partial_nucleated(df)
    pn['post_patch'] = 'Nuc-low seal'
    ns = no_seal(df)
    ns['post_patch'] = 'No-low seal'
    ec = entire_cell(df)
    ec['post_patch'] = 'Entire cell'
    return  nu, oo, pn, ns, ec

def concat_df(a, b, c, d, e):
    frames = (a, b, c, d, e)
    df = pd.concat(frames)
    return df

def postpatch_reclass(df):
    return concat_df(*reclassify(df))

#df = postpatch_reclass(df)

In [4]:
df = pd.read_csv('C:/users/kumar/allen-institute/personal-projects/csv/patch_seq_log_mouse.csv')
#parse_dates=['Date'])

list_initial = ['Date',
                'File',
                'User',
                'cell type',
                'Post patch?',
                'Post patch pipette R',
#                'Nucleus sucked in?',
                'Time spent extracting cytosol',
                'Time spent retracting pipette',
                'patch duration',
#                'Lims tube id',
#                'Well ID #',
                'PCR cycles',
                'SM_QC_PF']
df = df[list_initial]
df.head()

,Date,File,User,cell type,Post patch?,Post patch pipette R,Time spent extracting cytosol,Time spent retracting pipette,patch duration,PCR cycles,SM_QC_PF
0,170104,Gad2-IRES-Cre;Ai14-292660.07.01.01,P1,tdt+,Outside-Out,NaN,2.3,NaN,11.78333333,18.0,pass
1,170104,Gad2-IRES-Cre;Ai14-292660.07.01.02,P1,tdt+,No-Seal,NaN,2.316666667,NaN,12.21666667,18.0,pass
2,170104,Gad2-IRES-Cre;Ai14-292660.07.01.03,P1,tdt+,No-Seal,NaN,2.45,NaN,11.78333333,18.0,pass
3,170104,Gad2-IRES-Cre;Ai14-292660.08.01.01,P1,tdt+,No-Seal,NaN,2.316666667,NaN,12.45,18.0,pass
4,170104,Gad2-IRES-Cre;Ai14-292660.08.01.02,P1,tdt+,Outside-Out,NaN,2.35,NaN,12.36666667,18.0,pass


In [5]:
#Changing to string to add 20 in front of strings within date column
df['Date'] = df['Date'].astype('str')
df['Date'] = '20' + df['Date']
df['Date'] = pd.to_datetime(df['Date']) #Converting date to YYYY-MM-DD

df = df.set_index('Date') #Set date column as index column
df.sort_index(inplace = True) #Sort the date column just in case
df = df['2017-10-02':] #Choosing start & end date(Production date)

In [6]:
df.head()

,File,User,cell type,Post patch?,Post patch pipette R,Time spent extracting cytosol,Time spent retracting pipette,patch duration,PCR cycles,SM_QC_PF
Date,,,,,,,,,,
2017-10-02,Gad2-IRES-Cre;Ai14-350672.06.02.01,PA,tdt+,nucleus_absent,10.2,0.333333333,2,11.86666667,21.0,pass
2017-10-02,Gad2-IRES-Cre;Ai14-350672.04.01.04,P9,tdt+,nucleus_absent,16.2,0.566666667,2.066666667,7.616666667,21.0,fail
2017-10-02,Gad2-IRES-Cre;Ai14-350672.04.01.03,P9,tdt+,nucleus_absent,4.7,0.516666667,1.766666667,5.416666667,21.0,fail
2017-10-02,Gad2-IRES-Cre;Ai14-350672.04.01.02,P9,tdt+,nucleus_present,930.0,0.35,2.2,6.5,21.0,fail
2017-10-02,Gad2-IRES-Cre;Ai14-350672.03.01.01,P9,tdt+,nucleus_present,1000.0,0.283333333,1.766666667,7.283333333,21.0,pass


In [7]:
#Categories
df['User'] = df['User'].astype('category')
df['PCR cycles'] = df['PCR cycles'].astype('category')
df['SM_QC_PF'] = df['SM_QC_PF'].astype('category')

#Filtering 
df = df[df['cell type'] == 'Cre+'] #Shows only Cre+  
df = df[df['PCR cycles'] == 21] #Shows only 21 pcr cycles
df = df[df['SM_QC_PF'] == 'pass'] #Shows only pass
#df = df[df['SM_QC_PF'] != 'cnt'] #Shows everything except (cnt = control)

df = postpatch_reclass(df) #Adding post_patch column

C:\Users\kumar\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\kumar\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\kumar\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

In [8]:
df.head()

,File,User,cell type,Post patch?,Post patch pipette R,Time spent extracting cytosol,Time spent retracting pipette,patch duration,PCR cycles,SM_QC_PF,post_patch
Date,,,,,,,,,,,
2017-12-07,Chrna2-Cre_OE25;Ai14-362810.05.02.01,PA,Cre+,nucleus_present,2000.0,0.95,2.366666667,9.283333333,21.0,pass,Nuc-high seal
2017-12-07,Chrna2-Cre_OE25;Ai14-362810.05.01.02,PA,Cre+,nucleus_present,1000.0,0.766666667,2.483333333,8.566666667,21.0,pass,Nuc-high seal
2017-12-07,Chrna2-Cre_OE25;Ai14-362810.05.01.01,PA,Cre+,nucleus_present,2000.0,1.016666667,2.316666667,8.75,21.0,pass,Nuc-high seal
2017-12-07,Chrna2-Cre_OE25;Ai14-362810.05.02.02,PA,Cre+,nucleus_present,1000.0,0.433333333,3.116666667,18.55,21.0,pass,Nuc-high seal
2017-12-07,Cux2-CreERT2;Ai14-362878.04.01.02,P9,Cre+,nucleus_present,2000.0,0.466666667,2.75,7.083333333,21.0,pass,Nuc-high seal


In [9]:
df.tail()

,File,User,cell type,Post patch?,Post patch pipette R,Time spent extracting cytosol,Time spent retracting pipette,patch duration,PCR cycles,SM_QC_PF,post_patch
Date,,,,,,,,,,,
2018-04-10,Vip-IRES-Cre;Ai14-383085.05.01.01,P1,Cre+,entire_cell,4000.0,0.4,1.25,13.31666667,21.0,pass,Entire cell
2018-04-11,Ndnf-IRES2-dgCre;Slc17a6-IRES2-FlpO;Ai65-38393...,P9,Cre+,entire_cell,6.9,0.583333333,7.066666667,14.43333333,21.0,pass,Entire cell
2018-04-13,Gad2-IRES-Cre;Ai14-383745.03.01.02,P9,Cre+,entire_cell,0.0,0.583333333,5.6,12.05,21.0,pass,Entire cell
2018-04-25,Th-P2A-FlpO;Ai65F-385999.04.01.02,PB,Cre+,entire_cell,2.0,1.716666667,0.15,16.26666667,21.0,pass,Entire cell
2018-04-27,Gad2-IRES-Cre;Ai14-386058.02.01.03,P9,Cre+,entire_cell,7.0,0.566666667,0.666666667,7.85,21.0,pass,Entire cell


In [22]:
df1 = df[['post_patch']] #Creating a smaller dataframe